In [4]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.patheffects as path_effects
import geopy.distance

In [2]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.0/428.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
!wget https://lodmedia.hb.bizmrg.com/case_files/1113064/train_dataset_train_data_Mediawise.zip
!unzip /content/train_dataset_train_data_Mediawise.zip

--2024-07-06 16:25:18--  https://lodmedia.hb.bizmrg.com/case_files/1113064/train_dataset_train_data_Mediawise.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 95.163.53.117
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|95.163.53.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1768797 (1.7M) [application/x-zip-compressed]
Saving to: ‘train_dataset_train_data_Mediawise.zip’

train_dataset_train 100%[===================>]   1.69M  2.26MB/s    in 0.7s    

2024-07-06 16:25:20 (2.26 MB/s) - ‘train_dataset_train_data_Mediawise.zip’ saved [1768797/1768797]

Archive:  /content/train_dataset_train_data_Mediawise.zip
   creating: train_data_Mediawise/
  inflating: train_data_Mediawise/readme.md  
  inflating: __MACOSX/train_data_Mediawise/._readme.md  
  inflating: train_data_Mediawise/baseline.ipynb  
  inflating: __MACOSX/train_data_Mediawise/._baseline.ipynb  
  inflating: train_data_Mediawise/train_data.json  


In [8]:
df=pd.read_json('/content/train_data_Mediawise/train_data.json')

In [9]:
df=pd.concat([df,pd.json_normalize(df['targetAudience'])], axis=1)
#%%
df=df.drop(['targetAudience','id'], axis=1)

In [10]:
def mean_distance(points):  # среднее растояние от центра Москвы до точек
    count = 0
    coords = 55.753567, 37.621409
    sm = 0
    for i in points:
        count += 1
        new_coords = i['lat'], i['lon']
        sm += geopy.distance.geodesic(coords, new_coords).km
    return sm / count

In [11]:
df['new_dup'] = df.points.apply(lambda x: str(x))
#%%
df = df[~df.duplicated(subset=['new_dup', 'value', 'name', 'gender', 'ageFrom', 'ageTo', 'income'], keep='first')]
#%%
df.drop(columns=['new_dup'], inplace=True)
#%%
df = df[~df.hash.isin(['6e61cdda0b59d8e0', '72dcb197c082953c', 'bb353ebce20024f2', 'd39f3cd0f9d7d63c'])] # точки, находящиеся в Ростове-на-дону и Сургуте

<ipython-input-11-b9c51b9da6a6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['new_dup'], inplace=True)


In [12]:
df['mean_distance'] = df.points.apply(mean_distance)


In [16]:
#Делит отрезок на равные интервалы
def split_on_intervals(min_val, max_val, n):
  step = (max_val - min_val)/n
  intervals = [min_val+(step*x) for x in range(n+1)]
  return intervals

#Создает регионы для поля
def create_groups(x_intervals, y_intervals):
  groups = {}
  x_intervals = np.concatenate([[-np.inf], x_intervals, [np.inf]])
  y_intervals = np.concatenate([[-np.inf], y_intervals, [np.inf]])

  for x_i in range(len(x_intervals)-1):
    for y_i in range(len(y_intervals)-1):
      groups[f'x : {x_intervals[x_i]} - {x_intervals[x_i+1]} | y : {y_intervals[y_i]} - {y_intervals[y_i+1]}'] = 0

  return groups
#%%
#Сортирует точки по регионам
def sort_on_groups(x_vals, y_vals, x_intervals, y_intervals, groups, only_vals = False):

  for x, y in zip(x_vals, y_vals):
    for x_i in range(len(x_intervals)-1):
      for y_i in range(len(y_intervals)-1):
        if ((x_intervals[x_i] <= x < x_intervals[x_i+1]) and (y_intervals[y_i] <= y < y_intervals[y_i+1])):
          groups[f'x : {x_intervals[x_i]} - {x_intervals[x_i+1]} | y : {y_intervals[y_i]} - {y_intervals[y_i+1]}'] += 1


  if only_vals:
    return list(groups.values())

  return groups
#%%
def create_dataset(config, df):
  x_intervals = split_on_intervals(config['min_xval'], config['max_xval'], config['x_ngroups'])
  y_intervals = split_on_intervals(config['min_yval'], config['max_yval'], config['y_ngroups'])

  groups = create_groups(x_intervals, y_intervals)

  groups_values=[]

  for i in range(len(df)):
    g = df.iloc[i]
    points = np.array([[float(x['lat']), float(x['lon'])] for x in g['points']])


    group_values = sort_on_groups(points[:,0], points[:,1], x_intervals, y_intervals, groups.copy(), only_vals = True)
    groups_values.append(group_values)


  groups_values = np.array(groups_values)

  for i in range(len(groups.keys())):
    groups[list(groups.keys())[i]]=groups_values[:,i]


  return groups


In [17]:
config = {'min_xval':55.55, 'max_xval':55.95, 'min_yval':37.3, 'max_yval':37.9, 'x_ngroups': 50, 'y_ngroups': 50}


In [18]:
dataset = pd.DataFrame(create_dataset(config, df))


In [19]:
from autogluon.tabular import TabularPredictor

In [21]:
X = dataset.copy()
X = X.set_index(df.index)
X['gender'] = df['gender']
X['value'] = df['value']
X['ageFrom'] = df['ageFrom']
X['ageTo'] = df['ageTo']
X['income'] = df['income']
X['mean_distance'] = df['mean_distance']

In [34]:
predictor = TabularPredictor(label='value', eval_metric='root_mean_squared_error', path='AutogluonModels/with_nn_3')

In [35]:
predictor.fit(X, time_limit = 3 * 3600, hyperparameters={'NN_TORCH': {}, 'GBM': {}, 'CAT': {}, 'XGB': {}, 'FASTAI': {}, 'XT': {}, 'RF': {}}, presets='best_quality',num_stack_levels=3, num_bag_folds=15, num_bag_sets=3)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.16 GB / 12.67 GB (80.1%)
Disk Space Avail:   74.63 GB / 107.72 GB (69.3%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=15, num_bag_sets=3
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stac